In [1]:
import pandas as pd

df = pd.read_csv('./Datasets/Pizza Party Sales.csv')

Viendo la información del dataset podemos ver que **no contiene nulos**. Además, seis de las siete columnas son de tipo *'object'*, mientras que la restante es de tipo *'float64'*

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49574 entries, 0 to 49573
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   id      49574 non-null  object 
 1   date    49574 non-null  object 
 2   time    49574 non-null  object 
 3   name    49574 non-null  object 
 4   size    49574 non-null  object 
 5   type    49574 non-null  object 
 6   price   49574 non-null  float64
dtypes: float64(1), object(6)
memory usage: 2.6+ MB


##### **Primera visualización del Dataset**

**Formato de los dato:** La columna **id*** cuenta con el año del pedido primero y luego un número identificatorio de seis dígitos (*AAAA-000000*). La columna **date** tiene un formato de fecha de día/mes/año (*DD/MM/AAAA*).La columna **time** tiene un formato de hora:minutos:segundos (*HH:MM:SS*). La columna **name** contiene una abreviatura del nombre de la pizza. La columna *size* contiene la medida de la pizza representado con una letra (Ej. M = 'Medium'). La columna **type** contiene la variedad de la pizza. La columna **price** contiene el precio final de la venta representado con un valor de tipo *flotante*.

In [13]:
df.sample(5)

,id,date,time,name,size,type,price
39727,2015-017171,18/10/2015,13:05:09,hawaiian,L,classic,16.50
17744,2015-007656,09/05/2015,13:10:51,the_greek,XL,classic,25.50
30755,2015-013326,10/08/2015,19:41:10,spinach_fet,M,veggie,16.00
35158,2015-015228,11/09/2015,22:02:02,southw_ckn,S,chicken,12.75
10833,2015-004656,19/03/2015,20:43:33,spin_pesto,S,veggie,12.50


**Duplicados en la columna 'id':** La columna *'id'* no tiene un valor único por cada fila; cada fila corresponde a uno de los productos vendidos, si en una misma venta se vende más de un producto entonces serán registradas en filas diferentes con un mismo identificador compartido.

In [14]:
df[df.duplicated('id', keep=False)]

,id,date,time,name,size,type,price
1,2015-000002,01/01/2015,11:57:40,classic_dlx,M,classic,16.00
2,2015-000002,01/01/2015,11:57:40,mexicana,M,veggie,16.00
3,2015-000002,01/01/2015,11:57:40,thai_ckn,L,chicken,20.75
4,2015-000002,01/01/2015,11:57:40,five_cheese,L,veggie,18.50
5,2015-000002,01/01/2015,11:57:40,ital_supr,L,supreme,20.75
...,...,...,...,...,...,...,...
49567,2015-021347,31/12/2015,21:14:37,bbq_ckn,M,chicken,16.75
49568,2015-021347,31/12/2015,21:14:37,peppr_salami,S,supreme,12.50
49569,2015-021348,31/12/2015,21:23:10,four_cheese,L,veggie,17.95
49570,2015-021348,31/12/2015,21:23:10,napolitana,S,classic,12.00
